# Predicting Preferred positions Fifa 18 Players *k*-NN

In this Notebook we're going to predict the first item from the variable preferred positions, using the *k*-NN algorithm. This dataset can be found on [Kaggle](https://www.kaggle.com/datasets/thec03u5/fifa-18-demo-player-dataset).

In [26]:
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #We need this to split the data

## Data set

Let's first look at the dataset and see which variables we can use.

In [27]:
df = pd.read_csv("CompleteDataset.csv")
df.head(30) #show a bit more of the dataset

C:\Users\Wessel\AppData\Local\Temp\ipykernel_29608\3258759003.py:1: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("CompleteDataset.csv")


,Unnamed: 0,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,3,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,4,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,R. Lewandowski,28,https://cdn.sofifa.org/48/18/players/188545.png,Poland,https://cdn.sofifa.org/flags/37.png,91,91,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,...,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,88.0
6,6,De Gea,26,https://cdn.sofifa.org/48/18/players/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,90,92,Manchester United,https://cdn.sofifa.org/24/18/teams/11.png,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,E. Hazard,26,https://cdn.sofifa.org/48/18/players/183277.png,Belgium,https://cdn.sofifa.org/flags/7.png,90,91,Chelsea,https://cdn.sofifa.org/24/18/teams/5.png,...,59.0,47.0,81.0,61.0,87.0,87.0,82.0,88.0,64.0,82.0
8,8,T. Kroos,27,https://cdn.sofifa.org/48/18/players/182521.png,Germany,https://cdn.sofifa.org/flags/21.png,90,90,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,...,76.0,72.0,87.0,82.0,81.0,81.0,77.0,80.0,78.0,77.0
9,9,G. Higuaín,29,https://cdn.sofifa.org/48/18/players/167664.png,Argentina,https://cdn.sofifa.org/flags/52.png,90,90,Juventus,https://cdn.sofifa.org/24/18/teams/45.png,...,51.0,46.0,71.0,52.0,84.0,79.0,87.0,82.0,55.0,87.0


## Data cleaning

Let's select the variables. I also need to drop the rows with NaN's in them. Unfortunately our *k*-NN algorithm won't work with NaN's. For now, let's just drop the rows with NaN's. I select the values according to the positions and the individual rating for the positions, to later predict their preferred position.

In [28]:
df = df[['CAM','CB','CDM','CF','CM','LAM','LB','LCB','LCM','LDM','LF','LM','LS','LW','LWB','RAM','RB','RCB','RCM','RDM','RF','RM','RS','RW','RWB','ST','Preferred Positions']]
df = df.dropna() #get rid of rows with empty cells
df['Preferred Positions'] = df['Preferred Positions'].str.split(" ").str[0] # This takes the first part of the split string, necessary when a footballer likes 2 or more positions.
df.head()
df['Preferred Positions'].value_counts()

CB     2705
ST     2277
CM     1977
CDM    1396
RM     1350
LM     1326
LB     1309
RB     1195
CAM    1125
RW      429
LW      427
CF      205
LWB     120
RWB     111
Name: Preferred Positions, dtype: int64

## Building the model

I will choose KNN. KNN is a supervised learning algorithm, and its useful for inputting missing values. It takes a grid with training data, and knows what group they belong to. Then when you add a new value, it will choose the Nearest Neighbor (hence the name) and put it near them, giving it a label. 

In [29]:
X = df[['CAM','CB','CDM','CF','CM','LAM','LB','LCB','LCM','LDM','LF','LM','LS','LW','LWB','RAM','RB','RCB','RCM','RDM','RF','RM','RS','RW','RWB','ST']] #create the X matrix

y = df['Preferred Positions'] #create the y-variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) #split the data, store it into different variables

X_train.head() #show the head of the training set

,CAM,CB,CDM,CF,CM,LAM,LB,LCB,LCM,LDM,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
8237,67.0,61.0,63.0,67.0,66.0,67.0,61.0,61.0,66.0,63.0,...,61.0,61.0,66.0,63.0,67.0,67.0,65.0,67.0,62.0,65.0
4987,58.0,68.0,69.0,55.0,64.0,58.0,65.0,68.0,64.0,69.0,...,65.0,68.0,64.0,69.0,55.0,57.0,50.0,54.0,65.0,50.0
5150,61.0,70.0,69.0,59.0,65.0,61.0,66.0,70.0,65.0,69.0,...,66.0,70.0,65.0,69.0,59.0,60.0,58.0,58.0,66.0,58.0
16236,49.0,51.0,51.0,50.0,49.0,49.0,56.0,51.0,49.0,51.0,...,56.0,51.0,49.0,51.0,50.0,53.0,49.0,52.0,56.0,49.0
14919,54.0,43.0,43.0,56.0,51.0,54.0,41.0,43.0,51.0,43.0,...,41.0,43.0,51.0,43.0,56.0,51.0,59.0,52.0,42.0,59.0


Let's use the *KNeighrborsClassifier* class from sklearn:

In [30]:
from sklearn.neighbors import KNeighborsClassifier #the object class we need

knn = KNeighborsClassifier(n_neighbors=5) #create a KNN-classifier with 3 neighbors
knn = knn.fit(X_train, y_train) #this fits the k-nearest neigbor model with the train data



In [31]:
knn.n_samples_fit_

11166

## Model evaluation

Let's start by calculating accuracy. As always, we do the evaluation on the test data.

In [32]:
knn.score(X_test, y_test) #calculate the fit on the *test* data

0.411199331383201

Accuracy is 41,1%. This is not very accurate, but considering there are 14 positions not too bad. Let's create a confusion matrix to evaluate precision and recall.

In [33]:
from sklearn.metrics import confusion_matrix
y_test_pred = knn.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix"
cm

array([[ 97,   2,  45,   7,  58,   2,  54,   8,   1,   1,  30,   5,   0,
         36],
       [  3, 672,  38,   0,  34,  41,   2,   0,   0,  23,   1,   0,   0,
          0],
       [ 27,  88, 141,   0, 130,  14,   7,   0,   0,   4,   7,   0,   0,
          0],
       [  9,   0,   0,   3,   4,   0,   9,   2,   0,   0,   6,   2,   0,
         14],
       [ 82,  44, 153,   7, 201,  15,  26,   1,   0,   5,  24,   0,   0,
          3],
       [  5, 112,  28,   1,  21, 141,  17,   1,   2,  76,  13,   0,   0,
          2],
       [ 67,   7,  30,  10,  37,  34,  77,  14,   1,  13,  62,   5,   0,
         42],
       [ 21,   0,   3,   0,  11,   0,  30,   8,   0,   0,  19,   6,   0,
         29],
       [  3,   5,   3,   0,   2,   8,   3,   0,   0,   6,   4,   0,   0,
          0],
       [  5,  98,  24,   0,  25, 135,  17,   1,   3,  61,   8,   0,   1,
          0],
       [ 73,   6,  27,   4,  36,  35,  62,  16,   0,  13,  67,  11,   0,
         45],
       [ 20,   1,   3,   3,   8,   4,  29, 

How do I know which row is which (survived or not survived)? There's an attribute for that in our model:

In [34]:
knn.classes_

array(['CAM', 'CB', 'CDM', 'CF', 'CM', 'LB', 'LM', 'LW', 'LWB', 'RB',
       'RM', 'RW', 'RWB', 'ST'], dtype=object)

This tells me that the first outcome is the "0" value (not survived) and the second outcome the "1" value (survived). If there was text in that column instead of 0/1 numbers, it would show the category text label.

Let's pretty print our table:

In [35]:
conf_matrix = pd.DataFrame(cm, index=['CAM_actual', 'CB_actual', 'CDM_actual', 'CF_actual', 'CM_actual', 'LB_actual', 'LM_actual', 'LW_actual', 'LWB_actual', 'RB_actual','RM_actual', 'RW_actual', 'RWB_actual', 'ST_actual'], columns = ['CAM_predicted', 'CB_predicted', 'CDM_predicted', 'CF_predicted', 'CM_predicted', 'LB_predicted', 'LM_predicted', 'LW__predicted', 'LWB_predicted', 'RB_predicted','RM_predicted', 'RW_predicted', 'RWB_predicted', 'ST_predicted']) #make a dataframe, put labels on rows (index) and columns 
conf_matrix

,CAM_predicted,CB_predicted,CDM_predicted,CF_predicted,CM_predicted,LB_predicted,LM_predicted,LW__predicted,LWB_predicted,RB_predicted,RM_predicted,RW_predicted,RWB_predicted,ST_predicted
CAM_actual,97,2,45,7,58,2,54,8,1,1,30,5,0,36
CB_actual,3,672,38,0,34,41,2,0,0,23,1,0,0,0
CDM_actual,27,88,141,0,130,14,7,0,0,4,7,0,0,0
CF_actual,9,0,0,3,4,0,9,2,0,0,6,2,0,14
CM_actual,82,44,153,7,201,15,26,1,0,5,24,0,0,3
LB_actual,5,112,28,1,21,141,17,1,2,76,13,0,0,2
LM_actual,67,7,30,10,37,34,77,14,1,13,62,5,0,42
LW_actual,21,0,3,0,11,0,30,8,0,0,19,6,0,29
LWB_actual,3,5,3,0,2,8,3,0,0,6,4,0,0,0
RB_actual,5,98,24,0,25,135,17,1,3,61,8,0,1,0


#### Accuracy and Precision

Let's start with the accuracy. We already calculated it with the *.score* method, but just to check.

For ease of use, and because I don't know how to do it better, I take the 5 first examples, CAM, CB, CDB, CF and CMD.
For each indivual one I will first check the precision (correct predictions devided by the number of predicions)

In [36]:
CAM_correct = 97
CB_correct = 672
CDM_correct = 141
CF_correct = 3
CM_correct = 201
CAM_total = 97+3+27+9+82+5+67+21+3+5+73+20+3+43
CB_total = 2+672+88+0+44+112+7+5+98+6+1+4+3
CDM_total = 45+38+141+0+153+28+30+3+3+24+27+3+1+4
CF_total = 7+3+7+1+10+4+3+9
CM_total = 58+34+130+4+201+21+37+11+2+25+36+8+3+5

print("The precision for CAM:", str(CAM_correct/CAM_total))
print("The precision for CB:", str(CB_correct/CB_total))
print("The precision for CDM:", str(CDM_correct/CDM_total))
print("The precision for CF:", str(CF_correct/CF_total))
print("The precision for CM:", str(CM_correct/CM_total))
print("___")
print("The total accuracy for these 5 positions:", ((CAM_correct+CB_correct+CDM_correct+CF_correct+CM_correct))/(CAM_total+CB_total+CDM_total+CF_total+CM_total))




The precision for CAM: 0.21179039301310043
The precision for CB: 0.6449136276391555
The precision for CDM: 0.282
The precision for CF: 0.06818181818181818
The precision for CM: 0.34956521739130436
___
The total accuracy for these 5 positions: 0.4253531882397862


Aha! That explains our accuracy of 41,1% from earlier. There is a wide variety of accuracies between the different positions, some scoring as low as 6% and others performing 10 times better.
If I were to do a manual check for all the values, I would probably end up closer and closer to the 41,1% from earlier, but I don't know the efficient way of doing that.

#### Recall

Now recall. This is the number of correctly predicted positions, divided by the total number of actual positions.

In [37]:
CAM_recall = CAM_correct/(97+2+45+7+58+2+54+8+1+30+5+36)
CB_recall = CB_correct/(3+672+38+34+41+2+23+1)
CDM_recall = CDM_correct/(27+88+141+130+14+7+4+7)
CF_recall = CF_correct/(9+3+4+9+2+6+2+14)
CM_recall = CM_correct/(82+44+153+7+201+15+26+1+5+24+3)

print("The recall for CAM:", str(CAM_recall))
print("The recall for CB:", str(CB_recall))
print("The recall for CDM:", str(CDM_recall))
print("The recall for CF:", str(CF_recall))
print("The recall for CM:", str(CM_recall))
print("___")
print("The average recall of these 5 examples is ", str((CAM_recall+CB_recall+CDM_recall+CF_recall+CM_recall)/5))

The recall for CAM: 0.2811594202898551
The recall for CB: 0.8255528255528255
The recall for CDM: 0.3373205741626794
The recall for CF: 0.061224489795918366
The recall for CM: 0.3582887700534759
___
The average recall of these 5 examples is  0.37270921597095086


So 37.2%. So again a bit worse than accuracy overall but not much. I'm just happy that the recall and precision are close to the accuracy score we got earlier to be honest. 

## Parameter setting

Finally, let's try out different settings for the most important parameter *k*. I'll use a for-loop to do a simple parameter grid search. I'll use a built-in function *classification_report* in sklearn to print out accuracy, precision and recall quickly.

In [38]:
from sklearn.metrics import classification_report

for i in range(1,11):
    knn_new = KNeighborsClassifier(n_neighbors = i) #make a new kNN model with i (1-10) neighbors
    knn_new = knn_new.fit(X_train, y_train) #fit new model on train data
    y_test_pred_new = knn_new.predict(X_test) #predict using new model, with test data
    print(f"With {i} neighbors the result is:")
    print(classification_report(y_test, y_test_pred_new)) #use a built-in function to print out accuracy, precision and recall


With 1 neighbors the result is:
              precision    recall  f1-score   support

         CAM       0.21      0.19      0.20       346
          CB       0.68      0.71      0.69       814
         CDM       0.26      0.27      0.26       418
          CF       0.11      0.14      0.13        49
          CM       0.36      0.38      0.37       561
          LB       0.29      0.26      0.27       419
          LM       0.16      0.17      0.17       399
          LW       0.10      0.10      0.10       127
         LWB       0.08      0.09      0.09        34
          RB       0.28      0.26      0.27       378
          RM       0.18      0.19      0.19       395
          RW       0.09      0.08      0.08       131
         RWB       0.04      0.03      0.03        36
          ST       0.65      0.64      0.64       679

    accuracy                           0.37      4786
   macro avg       0.25      0.25      0.25      4786
weighted avg       0.37      0.37      0.37     

c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


With 8 neighbors the result is:
              precision    recall  f1-score   support

         CAM       0.22      0.24      0.23       346
          CB       0.66      0.84      0.74       814
         CDM       0.28      0.32      0.30       418
          CF       0.05      0.02      0.03        49
          CM       0.35      0.41      0.38       561
          LB       0.37      0.39      0.38       419
          LM       0.19      0.19      0.19       399
          LW       0.13      0.06      0.08       127
         LWB       0.00      0.00      0.00        34
          RB       0.31      0.19      0.23       378
          RM       0.22      0.16      0.19       395
          RW       0.13      0.04      0.06       131
         RWB       0.00      0.00      0.00        36
          ST       0.71      0.79      0.75       679

    accuracy                           0.43      4786
   macro avg       0.26      0.26      0.25      4786
weighted avg       0.39      0.43      0.41     

c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


With 9 neighbors the result is:
              precision    recall  f1-score   support

         CAM       0.23      0.24      0.23       346
          CB       0.66      0.85      0.74       814
         CDM       0.28      0.32      0.30       418
          CF       0.07      0.02      0.03        49
          CM       0.36      0.42      0.39       561
          LB       0.37      0.41      0.39       419
          LM       0.20      0.19      0.19       399
          LW       0.12      0.05      0.07       127
         LWB       0.00      0.00      0.00        34
          RB       0.32      0.18      0.23       378
          RM       0.21      0.16      0.18       395
          RW       0.18      0.05      0.07       131
         RWB       0.00      0.00      0.00        36
          ST       0.70      0.80      0.75       679

    accuracy                           0.43      4786
   macro avg       0.26      0.26      0.26      4786
weighted avg       0.40      0.43      0.41     

c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


With 10 neighbors the result is:
              precision    recall  f1-score   support

         CAM       0.24      0.24      0.24       346
          CB       0.66      0.85      0.75       814
         CDM       0.28      0.31      0.30       418
          CF       0.11      0.02      0.03        49
          CM       0.37      0.44      0.40       561
          LB       0.37      0.41      0.39       419
          LM       0.22      0.22      0.22       399
          LW       0.17      0.06      0.08       127
         LWB       0.00      0.00      0.00        34
          RB       0.34      0.20      0.25       378
          RM       0.24      0.19      0.21       395
          RW       0.15      0.04      0.06       131
         RWB       0.00      0.00      0.00        36
          ST       0.69      0.81      0.75       679

    accuracy                           0.44      4786
   macro avg       0.27      0.27      0.26      4786
weighted avg       0.40      0.44      0.42    

c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Wessel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


So in general, the more neighbours, the more accurate the model becomes, climbing slowly up to 0.40, 0.44 and 0.42 scores for respectively precision, recall and f1-score.


## Overal thoughts and conclusion
Overal, I think that this model is not the most accurate thing in the world. With an average score of around 40% of the time, you might think that you're better off flipping a coin, but you'd have to flip a coin with 200 sides and it will land on the good preferred position 40% of the time. This is not a good analogy.
To correctly predict the value out of 14 positions proved difficult. As we can see from the manual calculations for the precision and accuracy of for example CB, the score is much higher and more accurate. If we filtered the data which  for example only allowed entries with more than 50 preferred positions, we would have a higher score.
I think I did most of it correctly, but I look forward to your feedback!
Wessel